So first things first. Make sure you complete the routine steps:
1. Enter your netID into the github repo and then clone the repo to your local computer
2. Create a new conda environment using: "conda create -n hw7 python"
3. Activate the new environment: "conda activate hw7"
4. There are 4 packages you will be needing for this homework: gym, matplotlib, numpy and pytest. 
   Install these packages using "pip install -r requirements.txt"
5. Now you are all set to start.

### Multi-armed Bandit

So lets implement the simplest reinforcement learning (RL) recipe: **Multi-Armed Bandit (MAB)**.
For getting a concrete idea about the problem begin by having a quick look at Chapter 2 of Sutton and Barto's book on RL: http://incompleteideas.net/book/RLbook2018.pdf. In summary, the two core concepts that this algorithm drives through are:

1. Given a hypothetical setting where you are repeatedly faced with making a choice from **k** different options (each with
different reward amounts), how do you choose the *one* sweet option everytime so that you maximize your reward over a period of time? 
2. Along the course of your learning the best possible action, how do you balance between **exploiting** what you have       learned so far (i.e. choosing the best action from your current reward value estimation of the k actions) and **exploring** what you may not have learned yet (with the gamble that it may improve your current estimates of rewards and 'open up new avenues')? Sounds philosophical eh?

Here, we will be using the *gym* library from OpenAI that provides some really nice ready-made and fully customizable learning environments for testing your own RL recipe (https://gym.openai.com/docs/). Lets begin by importing *gym*:
<div class="alert alert-block alert-info">
<b>ATTENTION:</b> You might need to: pip install gym
</div>


In [2]:
import gym

Either you can create your own custom environment (the one for testing MAB i.e. Slot Machines is already done for you in the script slot_machines.py). or load a pre-existing one. So let us familiarize with some of the common aspects of a simple *gym* environment:

In [3]:
# Frozen Lake is a 4 x 4 grid world where your bot navigates from start to goal
# by treading on thin ice. Every step that it takes is ridden with uncertainty:
# The ice in the next step could be hard enough to withstand the weight of your
# bot or it could crack under its weight! The objective of this gruesome game is
# to reach the goal position in minimum steps and of course avoid meeting a chilly
# fate in the dark, deep and cold water underneath.
# To learn more: https://reinforcement-learning4.fun/2019/06/16/gym-tutorial-frozen-lake/
import numpy as np
myenv = gym.make('FrozenLake-v0') 
myenv.seed(0) # for repeatability

# Every environment has an action space i.e. the actions you can take:
print("Number of possible actions in Frozen Lake = " + str(myenv.action_space.n))

# and a state space i.e. the different states in the environment:
print("Number of possible states in Frozen Lake = " + str(myenv.observation_space.n))

# You begin by resetting your environment to the intial state:
intial_state = myenv.reset()
print("Beginning in state = " + str(intial_state))

# Now you (actually your AI bot) wanders through the evironment...step by step
# Its time to reset the game when your steps onto thin ice and it cracks under
# its weight!
done = False
while done is False:
    # first choose your action:
    action_idx = np.random.randint(myenv.action_space.n)

    # Now perform this action:
    print("Taking action number " + str(action_idx) + "...")
    observation, reward, done, info = myenv.step(action_idx) 
    print("New state is = " + str(observation) + ", reward value = " + str(reward) + "\nTime to reset? " + str(done))

Number of possible actions in Frozen Lake = 4
Number of possible states in Frozen Lake = 16
Beginning in state = 0
Taking action number 3...
New state is = 0, reward value = 0.0
Time to reset? False
Taking action number 3...
New state is = 0, reward value = 0.0
Time to reset? False
Taking action number 3...
New state is = 1, reward value = 0.0
Time to reset? False
Taking action number 1...
New state is = 0, reward value = 0.0
Time to reset? False
Taking action number 2...
New state is = 0, reward value = 0.0
Time to reset? False
Taking action number 3...
New state is = 0, reward value = 0.0
Time to reset? False
Taking action number 0...
New state is = 4, reward value = 0.0
Time to reset? False
Taking action number 2...
New state is = 8, reward value = 0.0
Time to reset? False
Taking action number 3...
New state is = 4, reward value = 0.0
Time to reset? False
Taking action number 1...
New state is = 4, reward value = 0.0
Time to reset? False
Taking action number 1...
New state is = 5, r

Refer to the OpenAI website for details of the values returned by *myenv.step(action_index)*. Here is a brief summary: 

> 1. In this homework we will not need *info*.
> 2. *observation* is an environment-specific object representing your observation of the environment. For the purposes of this homework its the the new state value of the environement as a result of your action.
> 3. *reward* is...reward.
> 4. *done* states whether it’s time to reset the environment again. When *done* is true, its time to reset (because its game over! Or else your bot gets stuck in the deep, dark and cold waters...forever).

So to reset your environment (which pulls your AI back to the start of the game) you simply:

In [4]:
starting_state = myenv.reset()
print("Returned to intial state = " + str (starting_state))

Returned to intial state = 0


The Slot Machines environment is already written for you. For using this environment in your own testing script (and also for running the algorithm for your free responses) all you need to do is follow these steps carefully:

> 1. Copy the slot_machines.py script to your *gym\envs\classic_control* folder (For Anaconda users it should be somewhere like: *Anaconda3\Lib\site-packages\gym\envs\classic_control*).
> 2. Edit the __init__.py file in *gym\envs\classic_control* to auto import this environment when you import gym:
     **from gym_slot.envs.slot_machines import SlotMachines**
> 3. Register your newly added environment by editing the __init__.py file in *gym\envs* (scroll down to the 'Classic' section and add the following lines at the end of this section):
**register(
    id='SlotMachines-v0',
    entry_point='gym.envs.classic_control:SlotMachines',
)**

<div class="alert alert-block alert-info">
<b>ATTENTION:</b> Make sure to complete the above steps before running the next snippet. You might need to reload this notebook at this point to ensure the modifications to your gym module takes effect properly.
</div>

Now let us try loading the environment so as to ensure that everything so far is working as expected:

In [5]:
myenv = gym.make('SlotMachines-v0')
myenv.seed(0)

[0]

There should not be any error at this point if you have done everything correctly. Feel free to reach out through **CampusWire** in case you face any issues.

Now that we have everything setup, let us walk through the *fit* function of MAB. The key aspect of the MAB problem is that there is only one state:

In [6]:
print("Number of possible states in Slot Machines = " + str(myenv.observation_space.n))

Number of possible states in Slot Machines = 1


So all that you really care about is how to estimate the reward values for each possible action over a certain time (i.e. steps)
As per the pseudocode in page 32 of Sutton and Barto's book, begin by setting up two variables for your *fit* function:

Q = vector to store the current reward estimate for each possible action (all zeros at the beginning).

N = vector to store the number of steps taken with each possible action (all zeros at the beginning).

And where do you get the number of possible actions from? Scroll up to find the answer if you don't recall. 

<div class="alert alert-block alert-info">
<b>ATTENTION:</b> Always make it a habit to reset your environment at the beginning of all functions.
</div>

So your *fit* function has acess to two parameters:
1. steps / iterations
2. epsilon

Remember the two core concepts we talked about at the beginning of this notebook? Now it all makes sense right! Yup you got it:
1. Sutton's book says you iterate forever till you get your best estimate. Well, if you iterate forever, you will actually land up getting the actual reward values rather than their estimates. That's an irony! So lets be practical and iterate through a finite number of steps so that we can finish the homework before the deadline.

    So in each step of your *for* loop you do the following:
        a. Be greedy: choose the action for which you get the best estimated reward (Hint: use the vector Q to do this).
        b. Do you have a tie for the best action in the step a? Then be fair and break the tie randomly.
        c. Take the step in the environment and collect your reward. (You know how to do this right? Hint: Scroll up)
        d. Update the step count for the action you just took (Hint: use the vector N to do this).
        e. Now the crux of the algorithm, the one key line that defines it all...update your reward estimate as:
           Q[action_you_took] = Q[action_you_took] + (1/N[action_you_took]) * (reward - Q[action_you_took])
        f. As always don't forget to check the status of DONE! Or else your bot is doomed for eternity...
Your homework code requires you to do one additional thing. You need to record the mean reward every **(s = floor(steps / 100))** step. There are number of ways you could do that easily. One simple way is to keep a separate varaible **reward_record** to sum succesive rewards you get each step till you encounter **mod(i/s) == 0** where **i** is your *for* loop counter. At this point you store the amount of summed up reward so far (after dividing it by **s**) and reset **reward_record** to zero.


2. Wait. Where does **epsilon** come into all this? Your hunch was right...the other core concept of this algorithm: *To explore or to exploit...thats the question.* How do you decide? You don't. You just let *the slings and arrows of outrageous fortune* do it for you. Humor apart, **epsilon** (value ranging from 0-1) is the probability that your bot will not be greedy. So in the step **a** of the pseudocode above, instead of choosing the action with the current maximum estimated reward everytime, you choose an action randomly from all available actions with **epsilon** probability (yes by doing so you also give the action with maximum reward a fair chance of getting chosen after all...so democratic!). There are a number of ways you could do it, my personal favorite is: 
        

In [7]:
epsilon = 0.2
augur_says = np.random.choice(['to exploit','to explore'], 1, p=[1.0 - epsilon, epsilon])[0]
if augur_says == 'to exploit':
    print("Augur advices to be greedy and take the known path.")
else:
    print("Augur advices that the gods are with you in exploring the unknown.")

Augur advices to be greedy and take the known path.


Thats it! Your first RL recipe is almost ready. All that is left is to do the magic i.e. predict. If you have been able to weather the storm so far, predicting should be a breeze. Essentially all your bot needs to do is follow the best possible course as set by your fitted Q vector (state_action_values). Remember that for MAB, there is only one state:
1. Press reset.
2. Set **done** to False.
3. Initialize **states**, **actions** and **rewards** as empty lists (you fill them up as you sail through the environment).
4. While **done** is False:
   
       a. Select the action with the maximum reward.
       b. Perform that action
       c. Collect your reward.

Remember to apppend the **state**, **action** and **reward** values for each iteration above to the corresponding lists.
Thats it! Return the **states**, **actions** and **rewards** lists (after converting them to arrays) and you're done.

### Q-learning

As earlier, begin by giving Sutton and Barto's book on temporal difference learning a quick read (Chapter 6). Temporal difference is a class of a number of algorithms of which Q-learning is a member. Why they are called 'temporal difference' learners is based on the reward updation step as you shall see soon (step 5 of *fit* function below). For the purposes of completing the code, here is what you need to know:
1. Now you can no longer assume that there is only one state in your environment. Refer to the Frozen Lake problem above for an example. Every action takes you to a new state where taking the same action as in the previous state does not necessarily elicit the same reward. So instead of considering actions alone, you need to consider (state,action) pairs. Thats the key concept.
2. There is a difference in the reward learning step...we will deal with that in a while.

So lets dive in:
This time your *fit* function has access to 2 additional parameters, **discount** and **adaptive**. You will soon see how we will use these in our code...
as in MAB, begin by setting up the following two variables:

Q = a matrix to store the current reward estimate for each possible state-action pair (all zeros at the beginning).

N = a matrix to store the number of steps taken for each possible state-action pair (all zeros at the beginning).

So if A is the number of possible actions, and S is the number of possible states in the environment, each of the above two matrcies should be of size (S,A). By now you surely know how to get S and A right?

<div class="alert alert-block alert-info">
<b>ATTENTION:</b> As always, rememeber to press the RESET button at the beginning!
</div>

Then, for the *for* loop its pretty much the same game as in MAB (except for a few subtle differences as you will see):
1. As in MAB be (1 - epsilon)% greedy and epsilon% non-greedy in choosing your action. Only this time be wary of the state you are currently in i.e. best action is the action with the maximum estimated reward for the current state. (Hint: use the matrix Q). 
2. Do you have a tie for the best action in step 1? Then be fair and break the tie randomly.
3. Take the step in the environment and collect your reward. Also record the **state_after_action** your bot enters as a result of taking the action. (You know how to do this right? Hint: Scroll up)
4. Update your step count for the action you just took. Again, be wary of the state you are in. (Hint: use the matrix N)
5. Now the crux of the algorithm, the subtle difference from MAB...update your reward estimate as:
*Q[state_before_action, current_action] = Q[state_before_action, current_action] + (1.0 / N[state_before_action, current_action]) x (reward + **discount** x max(Q[state_after_action,:]) - Q[state_before_action, current_action])*. Very carefully note how we use the the *state before the current action* i.e. the **state from the previous iteration** and the *state after taking the current action* i.e. the **state you enter in the current iteration after performing the action**. Note how the reward updation is based on an error value measuring the difference in the estimate for the current step as compared to the previous step. Hence the name *temporal difference*.
6. Update the **state_before_action** using: **state_before_action = state_after_action**.
6. As always don't ignore DONE! Or else...

<div class="alert alert-block alert-warning">
<b>SUBTLETY ALERT:</b> What state is your bot at the beginning of the game? Hint: Did you press the RESET button at the beginning?
</div>

Ah yes, your homework requires you to record the mean reward every **(s = floor(steps / 100))** step. Follow the same lead as in MAB...

Finally, lets deal with **adaptive**. So you can rely on the *augur* (read *np.random.choice*) to decide when to **explore**  and when to **exploit**. But hey! Thats what the medieval people did right? So as society evolved it became self-reliant and learned to trust its own experiences in deciding things rather than resort to fantasy. Likewise, as your bot gains experience in the environment with increasing number of steps it takes, the estimates for the rewards for (state,action) pairs get better and better. So the strategy is to gradually shift from epsilon% exploration (and consequently [1 - epsilon]% exploitation) to 100% exploitation since you know your estimates are as good as it can get in finite time. One systematic way to do this is to introduce a hyper-parameter **progress** which is simply how many steps your bot has taken so far (as a fraction of the maximum number of steps it is supposed to take). For your convenience, your script has been set-up so that **progress** based **epsilon** is obtained simply by calling the function *_get_epsilon(**progress**)*. So in step 1 of the the Q-learning *fit* function, for every iteration, you always use the **epsilon** valued obtained from *_get_epsilon(**progress**)* where **progress** = i / steps (assuming *i* is your *for* loop counter).

Now at this stage, *predict* is an absolute no-brainer for you. As you guessed its the same as in MAB. 

<div class="alert alert-block alert-warning">
<b>SUBTLETY ALERT:</b> When predicting in Q-learning, remember that the action that yields the maximum reward depends on the current state. Hint: use the fitted Q matrix for this.
</div>